# Drag your GAN
:Interactive Point-based Manipulation on the
Generative Image Manifold

참고링크
+ youtube : https://www.youtube.com/watch?v=2oLWZe9M1LU
+ blog : https://kimjy99.github.io/%EB%85%BC%EB%AC%B8%EB%A6%AC%EB%B7%B0/draggan/
+ paper : https://arxiv.org/abs/2305.10973

## Contributions
+ 이미지 합성(control)에서 기존의 모델을 앞섰으며, 자연스러운 합성이 가능해졌다.

## 기존 방법들의 문제점
이미지합성 control에는 3가지의 역할을 수행할 수 있어야 한다.
1. 유연성 : 위치, 자세, 모양 등의 여러 공간속성을 제어
2. 정확성 : 당연하게도, 높은 정확도가 필요하고
3. 일반성 : 특정한 카테고리에 국한되지 않아야 한다.

하지만, 기존의 방식들은 위의 특징을 하나 혹은 두개밖에 만족하지 못했다.

+ Conditional GAN : 일반성을 만족 x
+ control on unconditional GAN : 정확성 만족 x
+ text-conditional Diffusion : 정밀한 제어  x


+ __Point-Tracking__(기존에 가장 많이 이용된 방법론)
    + 자율주행자동차 vision분야로 인해 발전된 기술로, 물체의 움직임에 대한 정보를 가진 ground truth optical flow를 NN으로 훈련하여 데이터를 합성한다.
    + 대표적으로, RAFT, PIPs 기술이 있다.

optical flow란?

- motion : 시간에 따른 픽셀의 움직임
(단위 : frame)

- motion field : 움직임이 발생한 모든점의 motion vector로 얻어낸 motion map
(= optical field)

- optical flow : optical field를 계산하기 위해 frame간에 차이로부터 픽셀값과 그 이동을 계산하여 추출해낸 값

즉, 시간이 지나며 물체가 움직인 변화량을 의미.

optical flow를 well-define하게 되면 motion field를 잘 얻어낼 수 있다.


__but, 본 논문의 접근법이 기존 방법의 성능을 모두 앞섰다.__

## Method

StyleGAN2 architecture를 이용한다.


<img src="https://drive.google.com/uc?id=1k3MNF_ZkPDgsHd9op_2J3M1bZ1i20wl1">

notations
+ $z,w \in \mathbb{R}^{512}$ : latent code
+ $I = G(w)$ : output image
+ $w^+ \in \mathbb{R}^{l \times 512}$ : 각 layer마다 다른 latent를 적용

이 중에서도 본 논문에서는 $w^+ \in \mathbb{R}^{l \times 512}$를 이용한다.  
(Image2StyleGAN 논문에서 제안한 method. 이를 사용했을 때, 학습된 데이터 이외에도 복원을 해주는 능력을 가졌다.)

<img src="https://drive.google.com/uc?id=1cSbIISBlqBCq2bwKmNlKgseqFdrGNM3G" height=300>

+ ${p_i=(x_{p,i}, y_{p,i})|i=1,2,...,n}$ : handle points
+ ${t_i=(x_{t,i}, y_{t,i})|i=1,2,...,n}$ : target points

__Goal__  
이미지의 semantic한 position까지 포함하여 위의 점을 옮기자!

이를 위해서는 이미지를 직접 다루는 것이아니라 feature space를 이용해야 했고, GAN은 여기로부터 이미지의 포인트 추적이 충분히 가능할 만큼 구분되어있음을 가정한다. 즉, disentanglement를 기반으로 이야기한다.

1. Motion Supervision : 사용자의 의도에 맞게 object를 아주 조금 이동시킨다. 그리고 이를 dynamic하게 반복하여 target까지 이동시킨다.
2. Point Tracking : 위의 방법에서 object가 변화하므로, handle point도 이에 맞춰 이동시켜줘야한다.(이동 전에는 코였는데 이후엔 볼에 가있을 수도 있다.)   
하지만, feature의 변화가 어떻게 일어날지 모르므로 이를 추적할 수 있어야 한다.

<img src="https://drive.google.com/uc?id=1E-z_sJKRKQhLJLDTzrTvLoAvEvSg7upS" height=350>

### Motion Supervision
StyleGAN2의 6번째 feature map __F__을 사용한다.(화질과 구분성의 trade-off에서 best였다고함)

이러한 __F__를 bilinear하게 upsampling하여 원본 image크기와 맞춰준다.   
그리고, 다음의 motion supervision loss를 적용한다.

$\mathcal{L} = \sum^n_{i=0}\sum_{q_i\in \Omega_1(p_i,r_1)}||\mathbf{F}(q_i) - \mathbf{F}(q_i + d_i)||_1 + \lambda||(\mathbf{F} - \mathbf{F_0}) \cdot (1 - \mathbf{M})||_1$

+ $d_i$ : $p_i$에서 $t_i$로의 unit vector (1 or 2의 크기)
+ $\Omega_1(p_i,r_1)$ : $p_i$에서의 $r_1$만큼의 circle space(feature patch)
+ $F_0$ : 시작 이미지일때의 feature map
+ $M$ : reconstruction mask(1 -> move / 0 -> fix)

즉, 이동시키고자 하는 위치만큼의 정보의 차이를 feature map 상에서 뽑아내어 loss로 이용하겠다는 의미이다. 또한, 이를 점 하나가 아닌 그 주변정보를 이용해 합산하여 책정한다.  
__back-prop할 때, $\mathbf{F}(q_i)$로는 흘려보내지 않았는데, point가 역으로 흘러가는 것을 방지하기 위함이었다.__

이러한 loss를 이용해 $w^+$의 값을 update해준다. 특별히, 모든 값을 바꾸는 것이아니라, style-mixing 기법에서 착안하여, 처음의 6개 layer에 대한 이들만 바꿔주었다.

### Point Tracking
앞서 말한 것처럼, point update를 하지 않으면, 기존에 의도한 object가 아닌 다른 물체를 옮기는 문제가 생길 수 있다.

기존 방법론인 optical flow estimation 등이 있으나 본 논문에서는 새로운 방법을 제안한다. 이 역시 GAN의 disentanglement한 성질 덕분이라고 한다.

__feature patch에 대한 nearest neighbor search__를 이용

$p_i^{(new)} := \underset{q_i\in \Omega_2(p_i, r_2)}{\operatorname{argmax}}||\mathbf{F}'(q_i) - f_i||_1$

+ $f_i = \mathbf{F}_0(p_i)$ : intial handle point

즉, 바뀌고 난 이후의 feature map point에서, 바뀌기 전 feature map 기준 가장 유사한 point로 이동시킨다!

## Experiments

<img src="https://drive.google.com/uc?id=1jxxKGl-KNpbOyi9dv7C5odsnQfMqYRZy">

real image에 대해서도 GAN inversion을 수행하여 바꾸었을 때 잘 수행되는 모습이다.

<img src="https://drive.google.com/uc?id=1ssdFCnBHEvbifbPZq0mBOljwGp8ZSze_" height=350>

기존방법들과의 정성적인 비교이다. 훨씬 적은 조작으로 원하는 목적을 좋은 결과로 달성할 수 있었다.

<img src="https://drive.google.com/uc?id=1DKZ9aOA6nmDgD5Q1N0P5cXuG_DFXUFQk" height=200>

기존의 sota방법과 비교했을때, 그리고 원래 방법과 논문의 방법을 함께 썼을 때보다, 논문의 방법만 사용했을 때 가장 퀄리티가 좋았다.

<img src="https://drive.google.com/uc?id=1jO3iAfS9ZakBS7zF0ys7_s7iTJ7usjLX" height=170>

원래 이미지를 얼마나 잘 reconstrution(보존)하는지이다.  
LPIPS는 이미지로부터 feature를 뽑아내서 평가한 결과값

<img src="https://drive.google.com/uc?id=1lZuOmgYhrk5ec1byjHf40z9tVulhcugS">